<a href="https://colab.research.google.com/github/saransarvesh/ALY-6110/blob/main/Module5_Final_Project%20error.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [ ]:
from google.colab import files
upload = files.upload()

Saving h.csv to h.csv


In [ ]:
## dataset reading
df = spark.read.csv('h.csv', inferSchema=True, header = True)

In [ ]:
df.show(5)

+-----+------+-------+---+-------+-----------+------------+------------------+--------------+--------+-----------------+----------+------------+-------------+----+-----------+-----------------------+------------------------+----------------+-----------+-----------+------------+-----------+-----------+-----------+-----------+------+-----------+-----------+---------+-----------+---------+----------+-----------+-----------+---------+---------------+-----------+-------------+-----------+---------------+------------+-------------+-----------+-----------+-------------+-----+-----------+-----------+----+---+
|group|    ID|outcome|age|gendera|        BMI|hypertensive|atrialfibrillation|CHD with no MI|diabetes|deficiencyanemias|depression|Hyperlipemia|Renal failure|COPD| heart rate|Systolic blood pressure|Diastolic blood pressure|Respiratory rate|temperature|      SP O2|Urine output| hematocrit|        RBC|        MCH|       MCHC|   MCV|        RDW|  Leucocyte|Platelets|Neutrophils|Basophils|Ly

In [ ]:
row=df.count()
row


1177

In [78]:
df1 = df.select(df['Renal failure'],df['Systolic blood pressure'],df['Blood potassium'],df['heart rate'],df['hematocrit'],df['Urine output'],df['Urea nitrogen'],df['Creatinine'],df['hypertensive'],df['diabetes'],df['deficiencyanemias'],df['Lactic acid'],df['EF'])

In [79]:
df1.show()

+-------------+-----------------------+---------------+-----------+-----------+------------+-------------+-----------+------------+--------+-----------------+-----------+---+
|Renal failure|Systolic blood pressure|Blood potassium| heart rate| hematocrit|Urine output|Urea nitrogen| Creatinine|hypertensive|diabetes|deficiencyanemias|Lactic acid| EF|
+-------------+-----------------------+---------------+-----------+-----------+------------+-------------+-----------+------------+--------+-----------------+-----------+---+
|            1|            155.8666667|    4.816666667|68.83783784|26.27272727|        2155|         50.0|1.958333333|           0|       1|                1|        0.5| 55|
|            0|                    140|           4.45|          0|      30.78|        1425|  20.33333333|1.122222222|           0|       0|                1|        0.5| 55|
|            1|            135.3333333|          5.825|72.31818182|       27.7|        2425|  33.85714286|1.871428571|       

In [80]:
## string to integer
from pyspark.sql.types import IntegerType
df1 = df1.withColumn("heart rate", df1["heart rate"].cast(IntegerType()))
df1 = df1.withColumn("Systolic blood pressure", df1["Systolic blood pressure"].cast(IntegerType()))
df1 = df1.withColumn("Urine output", df1["Urine output"].cast(IntegerType()))
df1 = df1.withColumn("Lactic acid", df1["Lactic acid"].cast(IntegerType()))

In [81]:
## replacing NA values with mean
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols = ['EF','heart rate'],
    outputCols = ["{}_imputed".format(a) for a in ['EF','heart rate'] ]
).setStrategy("mean")

In [82]:
## ## replacing NA values with median
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols = ['Systolic blood pressure','Blood potassium','hematocrit','Urine output','Urea nitrogen','Creatinine','Lactic acid'],
    outputCols = ["{}_imputed".format(a) for a in ['Systolic blood pressure','Blood potassium','hematocrit','Urine output','Urea nitrogen','Creatinine','Lactic acid'] ]
).setStrategy("median")

In [83]:
imputer.fit(df1).transform(df1).show()

+-------------+-----------------------+---------------+----------+-----------+------------+-------------+-----------+------------+--------+-----------------+-----------+---+-----------------------+-------------------+---------------------+------------------+-------------------------------+------------------+--------------------+
|Renal failure|Systolic blood pressure|Blood potassium|heart rate| hematocrit|Urine output|Urea nitrogen| Creatinine|hypertensive|diabetes|deficiencyanemias|Lactic acid| EF|Blood potassium_imputed|Lactic acid_imputed|Urea nitrogen_imputed|Creatinine_imputed|Systolic blood pressure_imputed|hematocrit_imputed|Urine output_imputed|
+-------------+-----------------------+---------------+----------+-----------+------------+-------------+-----------+------------+--------+-----------------+-----------+---+-----------------------+-------------------+---------------------+------------------+-------------------------------+------------------+--------------------+
|      

In [84]:
df.describe().show()

+-------+-------------------+------------------+-------------------+------------------+-------------------+-----------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+------------------+-------------------+------------------+-----------------------+------------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-----------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-----------------+------------------+
|summ

In [85]:
df1.describe().show()

+-------+------------------+-----------------------+-------------------+-----------------+-----------------+------------------+------------------+------------------+-------------------+------------------+-------------------+------------------+------------------+
|summary|     Renal failure|Systolic blood pressure|    Blood potassium|       heart rate|       hematocrit|      Urine output|     Urea nitrogen|        Creatinine|       hypertensive|          diabetes|  deficiencyanemias|       Lactic acid|                EF|
+-------+------------------+-----------------------+-------------------+-----------------+-----------------+------------------+------------------+------------------+-------------------+------------------+-------------------+------------------+------------------+
|  count|              1177|                   1161|               1177|             1164|             1177|              1141|              1177|              1177|               1177|              1177|       

In [86]:
df1.printSchema()

root
 |-- Renal failure: integer (nullable = true)
 |-- Systolic blood pressure: integer (nullable = true)
 |-- Blood potassium: double (nullable = true)
 |-- heart rate: integer (nullable = true)
 |-- hematocrit: double (nullable = true)
 |-- Urine output: integer (nullable = true)
 |-- Urea nitrogen: double (nullable = true)
 |-- Creatinine: double (nullable = true)
 |-- hypertensive: integer (nullable = true)
 |-- diabetes: integer (nullable = true)
 |-- deficiencyanemias: integer (nullable = true)
 |-- Lactic acid: integer (nullable = true)
 |-- EF: integer (nullable = true)



In [87]:
#library
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler, StandardScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [89]:
#combine columns
only_features=['heart rate','Systolic blood pressure','deficiencyanemias','Blood potassium','hematocrit','EF','Urine output','Urea nitrogen','Creatinine','hypertensive','diabetes','Lactic acid']
assembler = VectorAssembler(inputCols=only_features, outputCol='features')
feat_df1 = assembler.transform(df1).select('features')
feat_df1.show(6)

+--------------------+
|            features|
+--------------------+
|[68.0,155.0,1.0,4...|
|[0.0,140.0,1.0,4....|
|[72.0,135.0,1.0,5...|
|[94.0,126.0,0.0,4...|
|[67.0,156.0,1.0,4...|
|[74.0,118.0,1.0,4...|
+--------------------+
only showing top 6 rows



In [90]:
#Linear Regression
output = assembler.transform(df1)
#input vs output
finalized_data = output.select('features','Renal failure')
finalized_data.show()

+--------------------+-------------+
|            features|Renal failure|
+--------------------+-------------+
|[68.0,155.0,1.0,4...|            1|
|[0.0,140.0,1.0,4....|            0|
|[72.0,135.0,1.0,5...|            1|
|[94.0,126.0,0.0,4...|            0|
|[67.0,156.0,1.0,4...|            1|
|[74.0,118.0,1.0,4...|            1|
|[69.0,106.0,0.0,4...|            1|
|[84.0,141.0,1.0,4...|            0|
|[91.0,98.0,0.0,4....|            1|
|[75.0,122.0,0.0,3...|            0|
|[95.0,149.0,0.0,4...|            0|
|[65.0,103.0,1.0,4...|            0|
|[78.0,126.0,0.0,3...|            0|
|[65.0,112.0,0.0,4...|            1|
|[98.0,107.0,0.0,3...|            0|
|[73.0,159.0,0.0,4...|            1|
|[83.0,157.0,1.0,4...|            1|
|[64.0,113.0,1.0,4...|            1|
|[82.0,162.0,1.0,5...|            0|
|[70.0,112.0,1.0,4...|            1|
+--------------------+-------------+
only showing top 20 rows



In [91]:
#split train & test data
train_data,test_data = finalized_data.randomSplit([0.8,0.2])

In [93]:
#train model
regressor = LinearRegression(featuresCol='features',labelCol='Renal failure')
#learn to fit the model from train set
regressor = regressor.fit(train_data)

Py4JJavaError: ignored